para esta tarea se pensó primero en como cargar los datos para ello usaremos python, con pandas y numpy, además de eso el procesamiento como tal de los datos, no es complejo, sin embargo para hacerlo más interesante utilizaremos numba con python para realizarlo de manera paralela ya que recientemente hicimos un curso del mismo, algo ha resaltar que ya podemos anticipar es que los resultados vas a aparecer en desorden, además de que tal vez por la cantidad de datos el programa resulte más lento.

In [40]:
# Importamos las librerías
from numba import cuda
import pandas as pd
import numpy as np
print(cuda.detect())

Found 1 CUDA devices
id 0    b'NVIDIA GeForce RTX 3070'                              [SUPPORTED]
                      Compute Capability: 8.6
                           PCI Device ID: 0
                              PCI Bus ID: 8
                                    UUID: GPU-35b3c601-3795-7d80-d202-66c78dcbdd65
                                Watchdog: Enabled
                            Compute Mode: WDDM
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported
True


In [41]:
# Cargamos los datos
A = pd.read_csv('datos.csv')
A.head()

,Creditos,Nota
0,3,45.4
1,4,46.3
2,3,50.0
3,4,30.4


A Continuación se empezó a pensar en qué cosas se pueden realizar en paralelo a través de un kernel, por ello vamos a crear una función que nos devuelva la suma de todas las notas teniendo en cuenta los créditos y cuantas notas hay, osea la cantidad de créditos totales.

In [42]:
# funcion que sume los datos y nos diga cuantos datos hay
def preProcessing(data):
    total_creditos = data['Creditos'].sum()
    total_notas = (data['Creditos'] * data['Nota']).sum()
    return np.array([total_creditos], dtype=np.float64), np.array([total_notas], dtype=np.float64)

Ahora lo que si podemos paralelizar es el calculo final dados todos los posibles resultados de la nota de algoritmos, de forma que con numba vamos a utilizar cuda para definir nuestra kernel

In [43]:
@cuda.jit
def solution(notas, creditos, resultados, notaMateria):
    idx = cuda.threadIdx.x 
    calificacion = (idx + 1) * 0.5
    notas_ = notas[0] + (calificacion * 3)
    resultados[idx] = notas_ / (creditos[0] + 3)
    notaMateria[idx] = calificacion

Ahora como tal vamos a utilizar las funciones, para ello va a ser necesario cargar el resultado de preProcessing en la memoria de la tarjeta gráfica de forma que sea más eficiente el programa, además es ideal pues solo tenemos que hacer lecturas en el programa.

Además de esto y por error previamente se pensaba que en el kernel se podía hacer un print directamente, sin embargo es necesario igual crear un array donde vamos a cargar muestras respuestas, copiarlo a la gpu, y después devolverlo a la cpu, para ser exactos son dos arrays.

In [44]:
creditos, notas = preProcessing(A)
result = np.zeros(10, dtype=np.float32)
materias = np.zeros(10, dtype=np.float32)
d_cre = cuda.to_device(creditos)
d_not = cuda.to_device(notas)
d_res = cuda.to_device(result)
d_ind = cuda.to_device(result)

solution[1, 10](d_not, d_cre, d_res, d_ind)

resultado = d_res.copy_to_host()
notaMateria = d_ind.copy_to_host()

for i, res in enumerate(resultado):
    print(f'Nota materia {notaMateria[i]}: Nota final calculada: {res}')

Nota materia 0.5: Nota final calculada: 34.97058868408203
Nota materia 1.0: Nota final calculada: 35.05882263183594
Nota materia 1.5: Nota final calculada: 35.14706039428711
Nota materia 2.0: Nota final calculada: 35.235294342041016
Nota materia 2.5: Nota final calculada: 35.32352828979492
Nota materia 3.0: Nota final calculada: 35.411766052246094
Nota materia 3.5: Nota final calculada: 35.5
Nota materia 4.0: Nota final calculada: 35.588233947753906
Nota materia 4.5: Nota final calculada: 35.67647171020508
Nota materia 5.0: Nota final calculada: 35.764705657958984


d:\Documentos\GitHub\Algoritmos\.venv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
